# Parallel Configuration

This Jupyter notebook will walk you through how to use threads to execute commands that configure your experiment. 

## Import the FABlib Library

The `show_config` method shows what some environment variables relevant to your FABRIC user are set to. All necessary variables may be set [here](../../../fabric_examples/fablib_api/configure_environment/configure_environment.ipynb).

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

## (Optional): Query for Available Testbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
output = fablib.list_sites()

## Create the Experiment Slice

The following creates a set of 10 nodes with basic compute capabilities on random sites.


In [ ]:
slice_name="MySlice130"

#Create a slice
slice = fablib.new_slice(name=slice_name)

for i in range(4):
    # Add a node
    node = slice.add_node(name=f"Node{i}")

#Submit the Request
slice.submit()

## Observe the Slice's Attributes


In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()

## Configure the Nodes in Parallel

Using the Node object's built-in `execute_thread` method, several nodes may be configured in parallel, instead of configuring them one-at-a-time as a singular process. The `execute_thread` method also supports multithreading on a single node.

In [ ]:
config_command = "sudo yum install -q -y net-tools"

#Create execute threads
execute_threads = {}
for node in slice.get_nodes():
    print(f"Starting config on node {node.get_name()}")
    execute_threads[node] = node.execute_thread(config_command, output_file=f"{node.get_name()}.log")


#Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")

In [ ]:
for node in slice.get_nodes():
    stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s` && netstat -i')
    print(stdout)

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()